# 概要
tiktoken は、OpenAI が開発した トークナイザー（tokenizer） のライブラリで、GPT系のモデル（たとえば gpt-3.5-turbo や gpt-4）が入力テキストを処理する際に、どのようにトークンに分割するかを決定するために使われます。
URL(https://github.com/openai/tiktoken)

# install
```bash
pip install tiktoken
```


# よく使用される用途
- チャット履歴のトークン数を計測して、モデルの制限内に収める
- プロンプトの設計（token制限を意識）
- コスト計算（OpenAIのAPIはトークン数に応じて課金）


# textのトークン数を計算

普通の用途

In [ ]:
import tiktoken

def count_tokens(text: str, encoding_name: str = "cl100k_base") -> int:
    """
    指定したテキストのトークン数を計算する関数。

    Args:
        text (str): トークン数を数えたい文字列。
        encoding_name (str): 使用するトークナイザの名前（デフォルト: "cl100k_base"）。これは、GPT-4, 3.5用

    Returns:
        int: テキストのトークン数。

    使用例:
        >>> count_tokens("Hello World!")
        3
    """
    enc = tiktoken.get_encoding(encoding_name)
    tokens = enc.encode(text)
    return len(tokens)

system_prompt = "あなたは優秀なアシスタントです。AをBに変換してください。"
count_tokens(system_prompt)

高速化用

In [ ]:
import tiktoken

def get_encoding(encoding_name: str = "o200k_base") -> tiktoken.Encoding:
    """指定したエンコーディング名に基づいて、tiktokenのエンコーディングを取得する関数。"""
    return tiktoken.get_encoding(encoding_name)

def count_tokens(encoder: tiktoken.Encoding, text: str) -> int:
    """
    指定したテキストのトークン数を計算する関数。
    o200k_base: gpt-4o, gpt-4o-mini

    Args:
        encoder (tiktoken.Encoding): 使用するエンコーディング。
        text (str): トークン数を数えたい文字列。

    Returns:
        int: テキストのトークン数。
    Usage:
        >>> encoder = get_encoding("o200k_base")
        >>> count_tokens(encoder, "Hello World!")
        3
    """
    tokens = encoder.encode(text)
    return len(tokens)


system_prompt = "あなたは優秀なアシスタントです。AをBに変換してください。"
encoder = get_encoding()
count_tokens(encoder, system_prompt)

# dataset内のtoken数の分布の可視化の実例

- input: dataset class
- output: token数の分布の可視化

In [ ]:
import tiktoken
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def count_tokens_wrapper(text, encoding_name="cl100k_base"):
    """
    子プロセスで実行するため、各テキストのトークン数を計算するラッパー関数。
    プロセス毎にエンコーダを取得します。
    """
    enc = tiktoken.get_encoding(encoding_name)
    tokens = enc.encode(text)
    return len(tokens)

# ds['train'] から article を取り出してリストにする
texts = [item['article'] for item in ds['train']]

N_CPUS = 10
# ProcessPoolExecutor を使って並列処理を実行
with ProcessPoolExecutor(max_workers=N_CPUS) as executor:
    # executor.map は順序を保ったまま結果を返してくれるので、tqdm で進捗管理します
    tokens_list = list(tqdm(executor.map(count_tokens_wrapper, texts), total=len(texts)))

print(tokens_list[:10])


import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# tokens_list から中央値と最頻値を計算
median_val = np.median(tokens_list)
average_val = np.mean(tokens_list)
max_val = np.max(tokens_list)
min_val = np.min(tokens_list)
print(f"Median: {median_val}, Average: {average_val:.1f}, Max: {max_val}, Min: {min_val}")

# ヒストグラム描画
plt.figure(figsize=(10, 6))
sns.histplot(tokens_list, bins=100, kde=True)
plt.title("Token Length Distribution")
plt.xlabel("Token Length")
plt.ylabel("Frequency")

# 中央値と最頻値の垂直線を追加
plt.axvline(median_val, color='r', linestyle='--', linewidth=2, label=f'Median: {median_val}')
plt.axvline(average_val, color='g', linestyle='-.', linewidth=2, label=f'Average: {average_val:.1f}')

plt.legend()
plt.show()
